In [44]:
%matplotlib inline
import utils;
reload(utils)
from utils import *
from keras.layers.core import Dropout
from keras.layers.convolutional import Conv1D
from __future__ import division, print_function

In [2]:
model_path = "/Users/trinakarmakar/anaconda2/models/"
data_path = "/Users/trinakarmakar/anaconda2/data/"

In [3]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [5]:
idx_arr = sorted(idx, key = idx.get)
idx_arr[:10]

[u'the', u'and', u'a', u'of', u'to', u'is', u'br', u'in', u'it', u'i']

In [7]:
idx2word = {v: k for k, v in idx.iteritems()} #mapping from id to word

In [18]:
#download user reviews using keras code

path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')

(x_train, y_train), (x_test, y_test) = pickle.load(f)

In [19]:
len(x_train)

25000

In [20]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [21]:
idx2word[1238]

u'immediately'

In [22]:
#whole review from id to word
' '.join([idx2word[i] for i in x_train[0]])

u"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [23]:
y_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [25]:
vocab_size = 5000
train = [np.array([i if i <vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test =  [np.array([i if i <vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [26]:
lens = np.array(map(len, train))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [27]:
#need to make each sentence constant length
#This will create nice rectangular matrix feeding into model.
#for sentence shorted than max_len will be padded with space and longer
# will be truncated

seq_len = 500

train = sequence.pad_sequences(train, maxlen = seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [28]:
train.shape

(25000, 500)

In [29]:
test.shape

(25000, 500)

In [36]:
#create simple model

model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='relu')])

In [37]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(train, y_train, 
          validation_data=(test, y_test), nb_epoch=2, batch_size=128)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.8545 - acc: 0.5800 - val_loss: 0.4482 - val_acc: 0.8209

A CNN is likely to work better because it take advantage of ordered data
and we will use conv1d as words are sequence of 1d data.

In [46]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2, noise_shape=None, seed=None),
    Conv1D(64, 5, padding='same', activation='relu'),
    Dropout(0.2, noise_shape=None, seed=None),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7, noise_shape=None, seed=None),
    Dense(1, activation='sigmoid')])

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  from ipykernel import kernelapp as app


In [51]:
conv1.compile(loss = 'binary_crossentropy', 
              optimizer=Adam(), metrics=['accuracy'])

In [52]:
conv1.fit(train, y_train, validation_data=(test, y_test),
          nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 91s - loss: 0.4942 - acc: 0.7307 - val_loss: 0.2763 - val_acc: 0.8867

In [54]:
conv1.save_weights(model_path + 'conv1.h5')
conv1.load_weights(model_path + 'conv1.h5')

In [55]:
#Use pre trained vectors

def get_glove_datasets(dataset):
    """
    Download the requesed glove dataset from file.fast.ai
    and return a location that can be passed load_vectors.
    """
    md5sums = {'6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
    '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
    '6B.300d': '30290210376887dcc6d0a5a6374d8255'
              }
    glove_path = os.path.abspath('data/glove/results')
    
    %mkdir -p $glove_path
    
    return get_file(dataset,
                   'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [57]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
           pickle.load(open(loc+'_words.pkl', 'rb')),
           pickle.load(open(loc+'_idx.pkl', 'rb')))

In [59]:
vecs, words, wordidx = load_vectors(get_glove_datasets('6B.50d'))

80003072/80107627 [============================>.] - ETA: 0s  

In [62]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))
    
    for i in range(1, len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[1-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            #if we can't find the word in glove we randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))
    #This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    
    emb/=3
    
    return emb

In [63]:
emb = create_emb()

In [68]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, dropout=0.2,
             weights = [emb], trainable=False), #setting false for glove
    Dropout(0.25),
    Convolution1D(64, 5, padding='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  app.launch_new_instance()


In [69]:
model.compile(loss = 'binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [70]:
model.fit(train, y_train, validation_data=(test, y_test), 
          nb_epoch=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 105s - loss: 0.6183 - acc: 0.6516 - val_loss: 0.5155 - val_acc: 0.7554

In [71]:
model.layers[0].trainable = True
model.optimizer.lr = 0.01

In [72]:
model.fit(train, y_train, validation_data=(test, y_test),
         nb_epoch=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 105s - loss: 0.4057 - acc: 0.8173 - val_loss: 0.3972 - val_acc: 0.8235

In [73]:
model.save_weights(model_path+'glove50.h5')

In [74]:
#Multi size CNN
from keras.layers import Merge

In [76]:
graph_in = Input((vocab_size, 50))
convs = []
for fsz in range(3,6):
    x = Convolution1D(64, fsz, padding='same', activation='relu')(graph_in)
    x = MaxPooling1D()(x)
    x = Flatten()(x)
    convs.append(x)
out = Merge(mode='concat')(convs)
graph = Model(graph_in, out)

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [79]:
model.compile(loss = 'binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [80]:
model.fit(train, y_train, validation_data=(test, y_test),
         nb_epoch=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 127s - loss: 0.3264 - acc: 0.8618 - val_loss: 0.2921 - val_acc: 0.8787

In [82]:
model.layers[0].trainable = False
model.optimizer.lr = 1e-5
model.fit(train, y_train, validation_data = (test, y_test), nb_epoch = 2, batch_size = 64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 130s - loss: 0.1999 - acc: 0.9217 - val_loss: 0.2580 - val_acc: 0.8943

## LSTM

In [87]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True,
             W_regularizer=l2(1e-6), dropout=0.2),
    LSTM(100),
    Dense(1, activation='sigmoid')])

model.compile(loss = 'binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  app.launch_new_instance()
/Users/trinakarmakar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(5000, 32, mask_zero=True, embeddings_regularizer=<keras.reg..., input_length=500)`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.fit(train, y_train, validation_data=(test, y_test), 
          nb_epoch=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 1210s - loss: 0.4614 - acc: 0.7686 - val_loss: 0.3618 - val_acc: 0.8580